In [ ]:
import torch
import numpy as np
from pathlib import Path
import os

from nemo.util.plotting import plot_surface
from nemo.dem import DEM
from nemo.nemov2 import NEMoV2
from nemo.util.paths import project_root

device = "cuda" if torch.cuda.is_available() else "cpu"

%load_ext autoreload
%autoreload 2

In [ ]:
DEM_DATA = "LDEM"  # "LAC" or "LDEM"

if DEM_DATA == "LAC":
    dem = DEM.from_file("../../data/Moon_Map_01_0_rep0.dat")
else:
    tif_path = Path("../../data/Site01_final_adj_5mpp_surf.tif")
    dem = DEM.from_file(tif_path)

In [ ]:
dem.surface_plot()

In [ ]:
dem.data.shape

In [ ]:
xyz = dem.get_xyz_combined()
xy = torch.from_numpy(xyz[:, :2]).float().to(device)
z = torch.from_numpy(xyz[:, 2]).float().to(device)

print(f"Real LDEM data shape: xy={xy.shape}, z={z.shape}")
print(
    f"Coordinate ranges: X({xy[:, 0].min():.3f}, {xy[:, 0].max():.3f}), Y({xy[:, 1].min():.3f}, {xy[:, 1].max():.3f})"
)
print(f"Elevation range: Z({z.min():.3f}, {z.max():.3f})")

In [ ]:
nemo = NEMoV2(device=device)
nemo.from_config("../../configs/nemo.yaml")

In [ ]:
nemo.compute_scaling_parameters(xy, z)
nemo.fit(
    xy,
    z,
    lr=1e-3,
    max_epochs=5000,
    batch_size=20000,
    verbose=False,
    early_stopping=False,
)

In [ ]:
nemo.save_model("nemo_model.pth")

print(f"Model size: {os.path.getsize('nemo_model.pth') / 1024**2:.2f} MB")

print(f"DEM data type: {dem.data.dtype}")
print(f"DEM data size: {dem.data.size * 4 / 1024**2:.2f} MB")

In [ ]:
dem.data.size

In [ ]:
nemo.surface_plot(xy, dem.data)

In [ ]:
load_nemo = NEMoV2()
load_nemo.load_model("nemo_model.pth")

In [ ]:
load_nemo.predict_height(xy)

In [ ]:
pred_z = nemo.predict_height(xy)
pred_grid = dem.data.copy()
pred_grid[:, :, 2] = pred_z.detach().cpu().numpy().reshape(dem.shape[:2])

plot_surface(pred_grid)

In [ ]:
pred_z

In [ ]:
plot_surface(dem.data)

In [ ]:
dem.data.shape

In [ ]:
plot_surface(xyz.reshape(dem.data.shape))

# LDEM


In [ ]:
tif_path = Path("../../data/Site01_final_adj_5mpp_surf.tif")
dem = DEM.from_file(tif_path)

In [ ]:
xyz = dem.get_xyz_combined()
xy = torch.from_numpy(xyz[:, :2]).float().to(device)
z = torch.from_numpy(xyz[:, 2]).float().to(device)

print(f"Real LDEM data shape: xy={xy.shape}, z={z.shape}")
print(
    f"Coordinate ranges: X({xy[:, 0].min():.3f}, {xy[:, 0].max():.3f}), Y({xy[:, 1].min():.3f}, {xy[:, 1].max():.3f})"
)
print(f"Elevation range: Z({z.min():.3f}, {z.max():.3f})")

# Create new NEMoV2 instance
nemov2 = NEMoV2(device="cuda" if torch.cuda.is_available() else "cpu")

# Test fitting with conservative settings
print("\nFitting NEMoV2 to real LDEM data...")
nemov2.fit(
    xy,
    z,
    lr=1e-3,
    max_epochs=5000,
    batch_size=20000,
    verbose=False,
    early_stopping=False,
)

In [ ]:
from nemo.fit import DEMBicubic

In [ ]:
tif_path = Path("../../data/Site01_final_adj_5mpp_surf.tif")
dem = DEM.from_file(tif_path)
XYZ = dem.data
X = XYZ[:, :, 0]
Y = XYZ[:, :, 1]
Z = XYZ[:, :, 2]
x = X[0, :]
y = Y[:, 0]
fitter = DEMBicubic(x, y, Z)
print("Fit cubic spline to DEM")

X_flat = X.flatten()
Y_flat = Y.flatten()
Z_pred_flat = fitter(X_flat, Y_flat)
height_error = np.abs(Z_pred_flat - Z.flatten())
print(f"Height MAE: {height_error.mean():.5f} m")

In [ ]:
Z_pred_flat

In [ ]:
Z.flatten()